In [14]:
import torch
from torch import Tensor
import numpy as np
import math

## Training set and test set

In [15]:
def disk(nb_points) :
    radius = 1/np.sqrt(2*np.pi)
    inp = Tensor(nb_points,2).uniform_(0,1)
    ratio = torch.floor(torch.norm(inp,p=2,dim=1)/radius)
    target = 1-torch.clamp(ratio,min=0,max=1)
    t = Tensor(nb_points,2)
    for i in range(nb_points):
        if target[i] == 0:
            t[i,:] = Tensor([-1,1])
        elif target[i] == 1:
            t[i,:] = Tensor([1,-1])    
    return inp, t

nb_points = 1000
# create train set and respective labels
train_input , train_target = disk(nb_points)
# create test set and respective labels
test_input , test_target = disk(nb_points)

## Simple structure

In [16]:
class Module ( object ) :
    def forward ( self , * input ) :
        raise NotImplementedError
        
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
        
    def param ( self ) :
        return []

#### Activation functions

In [17]:
def tanh(x):
    return x.tanh()

def dtanh(x):
    return 1-torch.tanh(x).pow(2)

In [18]:
def relu(x):
    return max(0,x)

def drelu(x):
    if x>0:
        out = 1
    elif x<=0:
        out = 0
    return out

class Relu(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,input):
        self.input = input
        return relu(input)
    
    def backward(self,output):
        return drelu(input)*output #to check, probably incorrect
    
    # here you need to add "def param" too

#### Loss functions

In [19]:
def loss(v, t):
    return (v - t).pow(2).sum()

def dloss(v, t):
    return 2 * (v - t)

## Linear Module

In [20]:
class Linear(Module):
    # in_features: size of each input sample
    # out_features: size of each output sample
    # bias: If set to False, the layer will not learn an additive bias. Default: ``True``

    # Attributes:
    # weight: the learnable weights of the module of shape (out_features x in_features)`
    # bias:   the learnable bias of the module of shape `(out_features)`

    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weights = Tensor(out_features,in_features).normal_(0,1e-1)
        self.bias = Tensor(out_features).zero_()
        self.dl_dw = Tensor(out_features,in_features).zero_()
        self.dl_db = Tensor(out_features).zero_()
                
    def forward(self,input):   # input sarebbe la x
        self.input = input
        self.s = torch.mv(self.weights,input) + self.bias
        return self.s

    def backward(self, dl_ds):  
        dl_dx = self.weights.t().mv(dl_ds)   # the problem is here! 
        
        self.dl_dw.add_(dl_ds.view(-1, 1).mm(self.input.view(1, -1)))  
        self.dl_db.add_(dl_ds)
        return dl_dx     
    
    def update_param(self, eta):
        self.weights = self.weights - eta * self.dl_dw
        self.bias = self.bias - eta * self.dl_db
    
    def reset_param(self):
        self.dl_dw.zero_()
        self.dl_db.zero_()

## Activation Module

In [21]:
class Sigma(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,input):
        self.s = input
        return tanh(input)
    
    def backward(self, dl_dx):
        dl_ds = dtanh(self.s)*dl_dx
        return dl_ds
    
    def update_param(self, eta):
        pass 
    
    def reset_param(self):
        pass
        
        #return dtanh(input)*output    
        #return 4 * (output.exp() + output.mul(-1).exp()).pow(-2)

## Sequential Module

In [22]:
class Sequential(Module):
    def __init__(self, module_list):
        self.modules = module_list
        
    def forward(self, x):
        for module in module_list:
            x = module.forward(x)
            
        return x        
        
    def backward(self,dl_d):
        for module in reversed(module_list):
            dl_d = module.backward(dl_d)
    
    def update_param(self,eta):
        for module in module_list:
            module.update_param(eta)
    
    def reset_param(self):
        for module in module_list:
            module.reset_param()

## Loss Module

In [23]:
class Loss(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, input, target):
        self.input = input
        self.target = target
        return loss(input, target)
    
    def backward(self):
        return dloss(self.input, self.target)

## Framework

### Training

In [32]:
# Create my sequential list
# List of layers
input_size = 2
hidden_units = 25
output_size = 2

# three linear layers
fc1 = Linear(input_size,hidden_units)
fc2 = Linear(hidden_units,hidden_units)
fc3 = Linear(hidden_units,output_size)

# three activation layers
act1 = Sigma()
act2 = Sigma()
act3 = Sigma()

module_list = [fc1, act1, fc2, act2, fc3, act3]

# framework
model = Sequential(module_list)
model_loss = Loss()

# training set
train_input = train_input
train_target = train_target

# training parameters
lr = 0.001
nb_epochs = 200

# testing set
test_input = test_input
test_target = test_target

# store results
results = Tensor(train_input.size(0), 2, nb_epochs).zero_()

for k in range(0, nb_epochs):

    # TRAINING
    acc_loss = 0   # accumulated loss
    nb_train_errors = 0
    nb_test_errors = 0
    
    # zero the derivatives
    model.reset_param()
    
    # forward pass for all training samples
    for n in range(train_input.size(0)):
        x = train_input[n]
        t = train_target[n]
        x = model.forward(x)
        
        results[n,:,k] = x
        
        # compute the error
        pred = x.max(0)[1][0]  # the result is the index (0 or 1) of the position where the max value is
        if x[0]>-0.5:
            pred=0
        targ = train_target[n,:].max(0)[1][0]
        if targ != pred:
            nb_train_errors = nb_train_errors + 1 
               
        #loss
        ### forward
        loss_sample = model_loss.forward(input = x, target = t)
        acc_loss += loss_sample
        ### backward
        grad_loss = model_loss.backward()
        
        # backward pass
        model.backward(grad_loss)  
        
        # TESTING
        x_test = test_input[n]
        t_test = test_target[n]
        x_test = model.forward(x_test)
        
        # compute the error
        pred_test = x_test.max(0)[1][0]  # the result is the index (0 or 1) of the position where the max value is
        #if x_test[0]>-0.5:
            #pred_test=0
        targ_test = test_target[n,:].max(0)[1][0]
        if targ_test != pred_test:
            nb_test_errors = nb_test_errors + 1 
        
        
    # update the derivatives
    model.update_param(eta=lr)
    
    
        
    print('epoch {:d} acc_train_loss {:.02f} acc_train_error {:.02f}% test_error {:.02f}%'.format(k+1,acc_loss,
                                                                            (100 * nb_train_errors) / train_input.size(0), (100 * nb_test_errors) / test_input.size(0) ))

epoch 1 acc_train_loss 1882.36 acc_train_error 86.80% test_error 12.20%
epoch 2 acc_train_loss 1038.44 acc_train_error 13.20% test_error 12.20%
epoch 3 acc_train_loss 1035.40 acc_train_error 13.20% test_error 12.20%
epoch 4 acc_train_loss 1031.37 acc_train_error 13.20% test_error 12.20%
epoch 5 acc_train_loss 1025.86 acc_train_error 13.20% test_error 12.20%
epoch 6 acc_train_loss 1018.06 acc_train_error 13.20% test_error 12.20%
epoch 7 acc_train_loss 1006.58 acc_train_error 13.20% test_error 12.20%
epoch 8 acc_train_loss 989.23 acc_train_error 13.20% test_error 12.20%
epoch 9 acc_train_loss 963.67 acc_train_error 13.20% test_error 12.20%
epoch 10 acc_train_loss 933.80 acc_train_error 13.20% test_error 12.20%
epoch 11 acc_train_loss 913.37 acc_train_error 13.20% test_error 12.20%
epoch 12 acc_train_loss 896.60 acc_train_error 13.20% test_error 12.20%
epoch 13 acc_train_loss 886.09 acc_train_error 13.20% test_error 12.20%
epoch 14 acc_train_loss 874.54 acc_train_error 13.20% test_error 1

KeyboardInterrupt: 

In [31]:
results[0:20,:,12]


-0.6980  0.6937
-0.7356  0.7264
-0.7780  0.7881
-0.6739  0.6615
-0.5537  0.4859
-0.7066  0.7045
-0.6354  0.6057
-0.7939  0.8064
-0.7154  0.6998
-0.7791  0.7906
-0.7783  0.7868
-0.6856  0.6788
-0.6580  0.6389
-0.5224  0.4432
-0.6117  0.5637
-0.7450  0.7392
-0.7885  0.7985
-0.7767  0.7885
-0.6220  0.5825
-0.6924  0.6730
[torch.FloatTensor of size 20x2]

In [55]:
x = Tensor(2,3).zero_()
print(x)


In [85]:
x[0]

-0.9400811791419983